Name: Soleil Umwiza

Student number: 4386019

In this notebook i am going to use the sklearn.ensemble. Ref: RandomForestClassifier. (n.d.-a). Scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.htmlalgorithm to classify emails as phishing or valid(safe). i will use a free and public dataset `phishing-and-ham-emails` downloaded from Ref: phishing_&_ham_emails.(2023, March 23). Kaggle. https://www.kaggle.com/datasets/mohamedouledhamed/phishing-and-ham-emails.  A phishing email can be recognized by a human by analyzing it's `Subject` and/or `Body text`. And since the subject and body are in unstructured free text format, Natural Language Processing with NLTK for `words tokenization` and `stemming` is used, Ref:  CH03.RSt2. (n.d.). https://www.nltk.org/book/ch03.html.

First, the versions of the required libraries are shown. It always wise to report the versions of the libraries used so that in case problems arise in the future, one can still go back to a state in which the notebook worked.

In [1]:
import re
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')
import warnings

warnings.filterwarnings('ignore')



print("scikit-learn version:", sklearn.__version__)     # 1.1.3
print("pandas version:", pd.__version__)            # 1.5.1
print("seaborn version:", sns.__version__)   



scikit-learn version: 1.3.0
pandas version: 1.5.3
seaborn version: 0.12.2


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\csten\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 📦 Data provisioning

As it is mentioned in the introduction, the datasets are downloaded from [Kaggle Phishing and ham emails](https://www.kaggle.com/datasets/mohamedouledhamed/phishing-and-ham-emails). I have 2 datasets, one for `phishing emails` which we call `df1` and the other for `Valid/safe emails` which we call `df2`. Below I am reading both datasets and print their shapes, we can see that they have the same number of columns/features.

In [2]:
df1 = pd.read_csv("phishing.csv", index_col=0)
df2= pd.read_csv("data_extracted_easy_ham.csv", index_col=0)
print(df1.shape)
print(df2.shape)


(2239, 12)
(2551, 12)


# Exploring the data

## 📃 Sample the data

Below I am viewing both datasets to have an impression of the data and see if indeed the columns names matches. For that I am printing 5 random rows.

In [3]:
df1.sample(5)

,Fullname,To,Subject,Body,Date,IP,Return_Path,Delivered_to,Content_type,Message_Id,X_uid,Content_Length
From,,,,,,,,,,,,
update@regions.com,Regions Bank,owner-openbsd-mobile-outgoing@example.com,Verify And Update Your Account,[],"Thu, 15 Feb 2007 08:22:40 +0500",77.110.147.175,update@regions.com,user@login.example.com,"multipart/alternative;\n\tboundary=""86BEDA..22...",hu6m2l--lkf87y-8$0p$g$-$b0u-9@ei1ng.1vh7,827.0,NaN
contact@amazon.com,<contact@amazon.com,user@example.com,Private Message From Amazon Accounts Managemen...,"Dear Amazon® member, It has come to our attent...","Wed, 07 Dec 2005 12:24:52 +0100",NaN,nobody@albergue.hospedajededicado.com,user@example.com,text/html,E1EjxPo-0006VN-Qa@albergue.hospedajededicado.com,250.0,NaN
aw-confirm@eBay.com,aw-confirm@eBay.com,undisclosed-recipients:;,eBay Listing confirmed. Sell another item now!,eBay sent this message Listing confirmed. Sell...,"Tue, 8 Aug 2006 19:24:16 -0700",66.125.41.182,aw-confirm@eBay.com,user@login.example.com,"text/html;\n\tcharset=""Windows-1251""",20060809020300.EB6E76CD0F6@mail.example.org,14.0,NaN
aw-confirm@eBay.com,eBay <aw-confirm@eBay.com,user@example.com,eBay Suspension - User Agreement - Abusing eBay,eBay sent this message to your registered emai...,"Tue, 05 Dec 2006 10:01:54 -0600",NaN,nobody@server.hostfit.com,user@login.example.com,text/html,E1Grcjy-0008IA-DT@server.hostfit.com,479.0,NaN
corporateclients.refx328051458485x.cm@nationalcity.com,National City,user <user@example.com>,National City corporate customer service: conf...,[],"Wed, 13 Jun 2007 11:02:38 -0400 (EDT)",120.64.100.191,corporateclients.refx328051458485x.cm@national...,user@example.com,"multipart/alternative;\n\tboundary=""--0MGI_RTD...",20070613150240.36C8E16243@mail.example.com,NaN,3677.0


In [4]:
df2.sample(5)

,Fullname,To,Subject,Body,Date,IP,Return_Path,Delivered_to,Content_type,Message_Id,X_uid,Content_Length
From,,,,,,,,,,,,
oberman@es.net,Kevin Oberman,exmh-users@example.com,Re: Sorting,--Multipart_Tue_Sep_10_08:56:11_2002-1\nConten...,"Tue, 10 Sep 2002 08:56:47 -0700",127.0.0.1,exmh-users-admin@example.com,yyyy@localhost.example.com,"multipart/mixed; boundary=""Multipart_Tue_Sep_1...",20020910155647.0B7FF5D04@ptavv.es.net,NaN,NaN
cdale@techmonkeys.net,CDale <cdale@techmonkeys.net,fork@example.com,boycotting yahoo,Because of this:\nhttp://hrw.org/press/2002/08...,"Tue, 17 Sep 2002 21:41:40 -0500 (CDT)",NaN,fork-admin@xent.com,yyyy@localhost.example.com,TEXT/PLAIN; charset=US-ASCII,Pine.LNX.4.44.0209172137100.5112-100000@isolne...,NaN,NaN
eugen@leitl.org,Eugen Leitl <eugen@leitl.org,<transhumantech@yahoogroups.com>,Making a mesh on the move,http://www.guardian.co.uk/Print/0%2c3858%2c448...,"Tue, 3 Sep 2002 08:05:27 +0200 (CEST)",NaN,fork-admin@xent.com,yyyy@localhost.example.com,TEXT/PLAIN; charset=ISO-8859-1,Pine.LNX.4.33.0209030804570.3124-100000@hydrog...,NaN,NaN
fork_list@hotmail.com,Mr. FoRK,<fork@example.com>,Am I This Or Not?,"Wow, talk about a pheenomeenon\n\n==\nhttp://w...","Mon, 30 Sep 2002 22:19:29 -0700",NaN,fork-admin@xent.com,yyyy@localhost.example.com,"text/plain; charset=""iso-8859-1""",DAV48S97K5Nvz3bGlab00007ca3@hotmail.com,NaN,NaN
felicity@kluge.net,Theo Van Dinter <felicity@kluge.net,Josh Hildebrand <josh@jedi.net>,Re: [SAtalk] redhat init.d script for spamd an...,--UHN/qo2QbUvPLonB\nContent-Type: text/plain; ...,"Fri, 6 Sep 2002 13:54:11 -0400",NaN,spamassassin-talk-admin@example.sourceforge.net,yyyy@localhost.example.com,multipart/signed; micalg=pgp-sha1;\n protoc...,20020906175411.GB18326@kluge.net,NaN,NaN


 From the sample above, we can see that the `From` column is treated as index, however, we can see that the values are not unique per row because the same sender may have sent emails more than once. Hence, I will reset the index and get a default index.

In [5]:
df1.reset_index(inplace=True)
df2.reset_index(inplace=True)

After resetting the indices, let's explore further the data. By looking at the columns information we see that for some columns we have too many missing values. For example columns "Content_Length and X_uid" have too many null values. However, we need to be carefull before we can decide if we can drop them, because it could be that the value being null is a contributing factor that an email is `valid` or `phishing`.

Let's verify them carefully, by going back to the `phishing` and `valid` before they are merged.


In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2239 entries, 0 to 2238
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   From            2239 non-null   object 
 1   Fullname        2236 non-null   object 
 2   To              2234 non-null   object 
 3   Subject         2219 non-null   object 
 4   Body            2213 non-null   object 
 5   Date            2239 non-null   object 
 6   IP              1404 non-null   object 
 7   Return_Path     2239 non-null   object 
 8   Delivered_to    2239 non-null   object 
 9   Content_type    2213 non-null   object 
 10  Message_Id      2239 non-null   object 
 11  X_uid           1816 non-null   float64
 12  Content_Length  699 non-null    float64
dtypes: float64(2), object(11)
memory usage: 227.5+ KB


In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   From            2551 non-null   object 
 1   Fullname        2535 non-null   object 
 2   To              2398 non-null   object 
 3   Subject         2551 non-null   object 
 4   Body            2551 non-null   object 
 5   Date            2551 non-null   object 
 6   IP              1703 non-null   object 
 7   Return_Path     2551 non-null   object 
 8   Delivered_to    2416 non-null   object 
 9   Content_type    2150 non-null   object 
 10  Message_Id      2551 non-null   object 
 11  X_uid           0 non-null      float64
 12  Content_Length  0 non-null      float64
dtypes: float64(2), object(11)
memory usage: 259.2+ KB


Below we are checking the counts of the null values of the 2 columns `X_uid` and `Content_Length` for each of the datasets. We can see that in the `valid emails` the two columns have no values at all, and in the `phishing` dataset almost 75% of the Content_Length and almost 20% of X_uid values are missing.

In [8]:
print('Phishing dataframe: Count of null values in the column X_uid is {}/{} rows'.format(df1['X_uid'].isnull().sum(),len(df1.index)));
print('Phishing dataframe: Count of null values in the column Content_Length is {}/{} rows\n'.format(df1['Content_Length'].isnull().sum(),len(df1.index)));

print('Valid email dataframe: Count of null values in the column X_uid is {}/{} rows' .format(df2['X_uid'].isnull().sum(),len(df2.index)));
print('Valid email dataframe: Count of null values in the column Content_Length is {}/{} rows\n' .format(df2['Content_Length'].isnull().sum(),len(df2.index)));

print('Valid email dataframe: Count of null values in the column IP is {}/{} rows' .format(df2['IP'].isnull().sum(),len(df2.index)));
print('Valid email dataframe: Count of null values in the column IP is {}/{} rows' .format(df2['IP'].isnull().sum(),len(df2.index)));

Phishing dataframe: Count of null values in the column X_uid is 423/2239 rows
Phishing dataframe: Count of null values in the column Content_Length is 1540/2239 rows

Valid email dataframe: Count of null values in the column X_uid is 2551/2551 rows
Valid email dataframe: Count of null values in the column Content_Length is 2551/2551 rows

Valid email dataframe: Count of null values in the column IP is 848/2551 rows
Valid email dataframe: Count of null values in the column IP is 848/2551 rows


##### Now the `Content_length`, `IP` and `X_uid` columns can be dropped due to too many missing values and also their less relevance in this analysis

In [9]:
df1=df1.drop(columns=['IP','Content_Length','X_uid'])
df2=df2.drop(columns=['IP','Content_Length','X_uid'])


# 🛠️ Preprocessing


### 🎯 Target variable

Since we have two separate datasets one for phishing emails and one for valid emails, we will first label the data by adding a new column `IsEmailPhishing` on both datasets which will indicate if an email is phishing or not. For phishing emails we put `1` and for valid emails `0`, the we will review the shapes of the datasets.


In [10]:
df1["IsEmailPhishing"] = 1

In [11]:
df1.IsEmailPhishing.value_counts()

1    2239
Name: IsEmailPhishing, dtype: int64

In [12]:
df2["IsEmailPhishing"] = 0


In [13]:
df2.IsEmailPhishing.value_counts()

0    2551
Name: IsEmailPhishing, dtype: int64

In [14]:
print(df1.shape)
print(df2.shape)

(2239, 11)
(2551, 11)


Since we have 2 datasets with differents target values, we will then merge both dataframes for further preprocessing and analysis

In [15]:
df = pd.concat([df1,df2], axis=0)
print(df.shape)

(4790, 11)


In [16]:
df.head(5)

,From,Fullname,To,Subject,Body,Date,Return_Path,Delivered_to,Content_type,Message_Id,IsEmailPhishing
0,emailconfirm@ebay.com,emailconfirm@ebay.com,NaN,eBay account verification!,"Dear eBay User , After fraud complaints from t...","Thu, 30 Oct 2003 08:52:16 +0200",user@example.com,user@example.com,"text/html;\n\tcharset=""Windows-1251""",200502211804.NAA02450@mail.example.com,1
1,aw45-confirm@ebay.com,Ebay Team,undisclosed-recipients: ;,eBay - verify your account information,Welcome Welcome to a community of sellers that...,"Fri, 1 Jul 2005 17:57:54 -0700",user@mail.example.com,user@example.com,"text/html;\n\tcharset=""Windows-1251""",200507020500.BAA12022@mail.example.com,1
2,service@paypal.com,PayPal <service@paypal.com,user@example.com,Notification of limited account access,Dear valued PayPal® member: PayPal® is committ...,"Wed, 17 Aug 2005 18:28:02 +0000 (GMT)",user@mail.example.com,user@example.com,text/html,20050817182802.703BC84B0A0@asia.ibbmonitor.com,1
3,service@intl.paypal.com,service@intl.paypal.com <service@intl.paypal.com,user@example.com,..PayPal Notification..: Update your information,PayPal You're Billing Information! Dear PayPal...,"Thu, 21 Sep 2006 04:46:36 +0200 (CEST)",wwwrun@s2.medienuniversum.com,user@login.example.com,text/html,20060921024636.9C7E3391672@s2.medienuniversum.com,1
4,service@paypal.com,service@paypal.com,undisclosed-recipients:;,We recently noticed one or more attempts to lo...,We recently noticed one or more attempts to lo...,"Thu, 21 Sep 2006 05:52:53 +0200",service@paypal.com,user@login.example.com,"text/html;\n\tcharset=""Windows-1251""",20060921034817.322E41C00245@mwinf1921.orange.fr,1


After merging the indices of the two merged datasets will be mixed up, because the `phishing` dataset had index from 0-2239 and `valid emails` dataset index 0-2551. Meaning that resulting merged dataset will have index from `0` to `2239` duplicate. So, resetting the index is needed to have sequential index, then after getting a new index, we will drop the mixed up one

In [17]:
df.reset_index(inplace=True)
df=df.drop(columns='index')

And we can verify that our new index is sequential by checking the last 5 rows

In [18]:
df.tail()

,From,Fullname,To,Subject,Body,Date,Return_Path,Delivered_to,Content_type,Message_Id,IsEmailPhishing
4785,rssfeeds@example.com,newscientist <rssfeeds@example.com,yyyy@example.com,Gene technique reveals human evolution,"URL: http://www.newsisfree.com/click/-3,871436...","Thu, 10 Oct 2002 08:04:25 -0000",rssfeeds@example.com,yyyy@localhost.example.com,text/plain; encoding=utf-8,200210100804.g9A84PK14188@dogma.slashnull.org,0
4786,rssfeeds@example.com,guardian <rssfeeds@example.com,yyyy@example.com,Go-ahead for new-style hospitals,"URL: http://www.newsisfree.com/click/-4,872399...","Thu, 10 Oct 2002 08:04:10 -0000",rssfeeds@example.com,yyyy@localhost.example.com,text/plain; encoding=utf-8,200210100804.g9A84AK14152@dogma.slashnull.org,0
4787,rssfeeds@example.com,newscientist <rssfeeds@example.com,yyyy@example.com,Malicious code hidden in email software,"URL: http://www.newsisfree.com/click/-3,871080...","Thu, 10 Oct 2002 08:04:26 -0000",rssfeeds@example.com,yyyy@localhost.example.com,text/plain; encoding=utf-8,200210100804.g9A84QK14191@dogma.slashnull.org,0
4788,rssfeeds@example.com,guardian <rssfeeds@example.com,yyyy@example.com,'Flexible' retirement gains ground,"URL: http://www.newsisfree.com/click/-4,872400...","Thu, 10 Oct 2002 08:04:10 -0000",rssfeeds@example.com,yyyy@localhost.example.com,text/plain; encoding=utf-8,200210100804.g9A84AK14155@dogma.slashnull.org,0
4789,rssfeeds@example.com,newscientist <rssfeeds@example.com,yyyy@example.com,Critical US satellites could be hacked,"URL: http://www.newsisfree.com/click/-3,870882...","Thu, 10 Oct 2002 08:04:26 -0000",rssfeeds@example.com,yyyy@localhost.example.com,text/plain; encoding=utf-8,200210100804.g9A84QK14194@dogma.slashnull.org,0


Let's recheck the sumary of the whole dataset after merging. Now we can see that we still have some missing values. Let's start `Feature Selection` process during which we will decide if a column can be considered in our analysis. 

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4790 entries, 0 to 4789
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   From             4790 non-null   object
 1   Fullname         4771 non-null   object
 2   To               4632 non-null   object
 3   Subject          4770 non-null   object
 4   Body             4764 non-null   object
 5   Date             4790 non-null   object
 6   Return_Path      4790 non-null   object
 7   Delivered_to     4655 non-null   object
 8   Content_type     4363 non-null   object
 9   Message_Id       4790 non-null   object
 10  IsEmailPhishing  4790 non-null   int64 
dtypes: int64(1), object(10)
memory usage: 411.8+ KB


### 💡 Feature selection

Since for analyzing phishing emails big focus is on the sender of email than the recipient, the columns `To` and `Delivered_to` are less relevant and will not have much added value on our analysis. The same apply to `Fullname`,`Message_Id`,`date` and `Return_Path` columns. `From` column can have added value in this analyzis, however, it will not be considered for this iteration.

The most relevant columns are `Subject`and `Body` since they contains words that will be tokenized and turned to features

Let's fill missing values with `None` on `Subject` and `Body` features

In [20]:
df['Subject'].fillna('None', inplace=True)
df['Body'].fillna('None', inplace=True)

###### Leaving out non-used columns

Let's merge `Subject` text with `Body` columns in a new column `email_text`since both represent the text of an email.

Getting a subset of our dataframe by taking only `email_text` and our target value `IsEmailPhishing` and create a new dataframe `emailText_with_type` and view first 5 rows.

In [21]:
emailText_with_type=df[['Subject','Body','IsEmailPhishing']]
emailText_with_type['email_text']=df['Subject'] + ' ' + df['Body']
emailText_with_type=emailText_with_type[['email_text','IsEmailPhishing']]
emailText_with_type.head()

,email_text,IsEmailPhishing
0,"eBay account verification! Dear eBay User , Af...",1
1,eBay - verify your account information Welcome...,1
2,Notification of limited account access Dear va...,1
3,..PayPal Notification..: Update your informati...,1
4,We recently noticed one or more attempts to lo...,1


##### Tokenization and stemming

For this Iteration, I will only consider the columns `Body` and `Subject` merged as `email_text` for the analysis.
For that I will perform tokenization together with Stemming of the email text using NLTK pre processing. Within the function `text_processing`, I am using regex library to remove all ponctuations. 

Ref: GeeksforGeeks. (2023, May 16). Removing stop words with NLTK in Python. https://www.geeksforgeeks.org/removing-stop-words-nltk-python/


In [22]:
from nltk.corpus import stopwords

In [23]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

ps = PorterStemmer()

stop_words = set(stopwords.words('english'))
 
# extracting only sequence of letters and numbers from the text, converts the words in word_tokens to lower case and then checks whether
#they are present in stop_words or not, return tokens without stopwords in a list of words
def text_preprocessing(dataset):
    tokens_list=[]
    for i in range(0,len(dataset)):
        text=re.sub('[^a-zA-Z0-9]',' ',dataset['email_text'][i])
        text=text.lower()
        text=text.split()
        text=[ps.stem(w) for w in text if not w in stop_words]
        text =' '.join(text)
        tokens_list.append(text)
    return tokens_list

### Text processing using NLTK

In [25]:
email_words=text_preprocessing(emailText_with_type)
emailText_with_type['email_words']=email_words

In [26]:
emailText_with_type.sample(5)

,email_text,IsEmailPhishing,email_words
540,Notification of Limited Account Access Februar...,1,notif limit account access februari 2007 paypa...
4536,Distributed.net cracks the RC5-64 cipher URL: ...,0,distribut net crack rc5 64 cipher url http boi...
2784,RE: Selling Wedded Bliss (was Re: Ouch...) Hah...,0,sell wed bliss ouch hah guess want everyon kno...
3727,"Re: [SAtalk] Tru64 compile of SA On Thu, 29 Au...",0,satalk tru64 compil sa thu 29 aug 2002 rick be...
853,"BoA: instructions for customer [, ]",1,boa instruct custom


##### Data Shuffling

Now since our dataset is ordered starting with phishing emails and ends with valid emails. To avoid that splitting our dataset may be unbalanced, I will shuffle the dataset using the `sample` function and save the shuffled dataset in a new variable. Then will reoder the columns so we have the target variable as a last column and reset the index.

During my reading I understood that Random Forest Classification model shuffles data during training, maybe this step is not needed, but I am not sure.

In [27]:
emailText_with_type= emailText_with_type.sample(frac=1)

In [28]:
emailText_with_type=emailText_with_type.reset_index(drop=True)

Changing order of the columns making the target column last and drop the `email_text` since we have a corpus column `email_wors`

Ref: How to change the order of DataFrame columns? (n.d.). Stack Overflow. https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns

In [29]:

cols = emailText_with_type.columns.tolist()
cols = cols[-1:] + cols[:-1]
emailText_with_type=emailText_with_type[cols]

In [30]:
emailText_with_type=emailText_with_type.drop(columns='email_text')

In [31]:
emailText_with_type.head()

,email_words,IsEmailPhishing
0,sadev live rule updat releas ye great mind thi...,0
1,wifi progress well look pretti much 99 sure mo...,0
2,paypal account review depart recent notic one ...,1
3,import notic dear lasal bank custom dear lasal...,1
4,import notic fifth third bank,1


Assigning feature and target to variables

In [32]:
X = emailText_with_type['email_words']
y = emailText_with_type['IsEmailPhishing']

### 🪓 Splitting into train/test

After assigning the feature and target to X and Y variables, I am going to split the train and test data. As per examples in class, I am using 20% for test data and 80% for train data. I also choose random_state of 80.

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=80)
print("There are in total", len(X), "observations, of which", len(X_train), "are now in the train set, and", len(X_test), "in the test set.")

There are in total 4790 observations, of which 3832 are now in the train set, and 958 in the test set.


# 🧬 Modelling
In this step only the train set is used to fit the model, which in this case uses a Random forest Classifier algorithm for classification named sklearn.ensemble. Ref: RandomForestClassifier. (n.d.-b). Scikit-learn. https://scikitlearn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html And after that the test set is used to calculate the model's accuracy, in other words how well it performs. Accuracy is a fraction where any value closer to 1 is considered better, and 1 itself (100% accurate) is usually impossible.

Since NLP doesn't understand textual data, the words need some how to be encorded in numbers. Here I am using CountVectorizer to encode our email texts(tokens) to a vector based on the count of each word that occurs in an email. Ref: Prashantrajkaggle. (2023, July 4). Spam message classification with NLP. Kaggle. https://www.kaggle.com/code/prashantrajkaggle/spam-message-classification-with-nlp/notebook



In [34]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=2000,ngram_range=(1,2))
X_train=cv.fit_transform(X_train).toarray()

In [35]:
X_test=cv.transform(X_test).toarray()

In [36]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier().fit(X_train,y_train)

In [37]:
y_pred=classifier.predict(X_test)
                              

In [38]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

accuracy_score(y_test,y_pred)

0.9895615866388309

In [39]:
confusion_matrix(y_test,y_pred)

array([[482,   5],
       [  5, 466]], dtype=int64)

# 🔬 Evaluation

Looking at the accuracy score we see that this model perform very well as it is predicting with almost 99% accuracy. We have some hyperparameters `max_features` and `ngram_range` used in the CountVectorizer that may playing a role in the performance of this model. However, myself I am not able to explain them at the moment(More reading needed). I got idea on this `Modeling` part from Prashantrajkaggle. Ref: Prashantrajkaggle. (2023, July 4). Spam message classification with NLP. Kaggle. https://www.kaggle.com/code/prashantrajkaggle/spam-message-classification-with-nlp/notebook

In [40]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       487
           1       0.99      0.99      0.99       471

    accuracy                           0.99       958
   macro avg       0.99      0.99      0.99       958
weighted avg       0.99      0.99      0.99       958

